# CS246 - Colab 9
## Studying COVID-19

### Setup

Let's setup Spark on your Colab environment.  Run the cell below!

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 198 kB 42.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=56ca91365bdcb8709c066481bc4fc9a2ab802766d5789ab6676c2be42432159d
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark
The following package was automatically installed and is no longer required:
  libnvidia-common-470
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 39 not upgraded.
Need to get 36.5 MB of archives.
After t

Now we authenticate a Google Drive client to download the files we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
id='1YT7ttUAafCjbVdm6obeHp1TWAK0rEtoR'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('time_series_covid19_confirmed_global.csv')

id='1YxEA5UQ2EFJ_9oLssM__Gs1ncVNufGNA'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('time_series_covid19_deaths_global.csv')

id='1CNxszuZTeIw-5cF5yrzKMZdb1qV0hSoy'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('time_series_covid19_recovered_global.csv')

If you executed the cells above, you should be able to see the dataset we will use for this Colab under the "Files" tab on the left panel.

Next, we import some of the common libraries needed for our task.

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

Let's initialize the Spark context.

In [5]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

You can easily check the current version and get the link of the web interface. In the Spark UI, you can monitor the progress of your job and debug the performance bottlenecks (if your Colab is running with a **local runtime**).

In [6]:
spark

If you are running this Colab on the Google hosted runtime, the cell below will create a *ngrok* tunnel which will allow you to still check the Spark UI.

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

### Data Loading

In this Colab, we will be analyzing the timeseries data of the Coronavirus COVID-19 Global Cases, collected by Johns Hopkins CSSE.

Here you can check a realtime dashboard based on this dataset: [https://www.arcgis.com/apps/opsdashboard/index.html?fbclid=IwAR2hQKsEZ3D38wVtXGryUhP9CG0Z6MYbUM_boPEaV8FBe71wUvDPc65ZG78#/bda7594740fd40299423467b48e9ecf6](https://www.arcgis.com/apps/opsdashboard/index.html?fbclid=IwAR2hQKsEZ3D38wVtXGryUhP9CG0Z6MYbUM_boPEaV8FBe71wUvDPc65ZG78#/bda7594740fd40299423467b48e9ecf6)

---



*   ```confirmed```: dataframe containing the cumulative number of confirmed COVID-19 cases, divided by geographical area
*   ```deaths```: dataframe containing the cumulative number of deaths due to COVID-19, divided by geographical area
*   ```recovered```: dataframe containing the cumulative number of recoevered patients, divided by geographical area

The data sets contain data entries for each day, representing the cumulative totals as of that day.







In [7]:
confirmed = spark.read.csv('time_series_covid19_confirmed_global.csv', header=True)
deaths = spark.read.csv('time_series_covid19_deaths_global.csv', header=True)
recovered = spark.read.csv('time_series_covid19_recovered_global.csv', header=True)

In [8]:
confirmed.printSchema()

root
 |-- Province/State: string (nullable = true)
 |-- Country/Region: string (nullable = true)
 |-- Lat: string (nullable = true)
 |-- Long: string (nullable = true)
 |-- 1/22/20: string (nullable = true)
 |-- 1/23/20: string (nullable = true)
 |-- 1/24/20: string (nullable = true)
 |-- 1/25/20: string (nullable = true)
 |-- 1/26/20: string (nullable = true)
 |-- 1/27/20: string (nullable = true)
 |-- 1/28/20: string (nullable = true)
 |-- 1/29/20: string (nullable = true)
 |-- 1/30/20: string (nullable = true)
 |-- 1/31/20: string (nullable = true)
 |-- 2/1/20: string (nullable = true)
 |-- 2/2/20: string (nullable = true)
 |-- 2/3/20: string (nullable = true)
 |-- 2/4/20: string (nullable = true)
 |-- 2/5/20: string (nullable = true)
 |-- 2/6/20: string (nullable = true)
 |-- 2/7/20: string (nullable = true)
 |-- 2/8/20: string (nullable = true)
 |-- 2/9/20: string (nullable = true)
 |-- 2/10/20: string (nullable = true)
 |-- 2/11/20: string (nullable = true)
 |-- 2/12/20: string (

### Your Task

We are aware of the stress we are all experiencing because of the still-ongoing pandemic and the fact that many of you have projects and exams due this week due to the lack of a finals week. As such, we decided to conclude our series of Colabs with a **lightweight task** -- given everything you have learned about Spark during the quarter, this Colab should take you just a few minutes to complete.

Consider the entries for May 1, 2021, in the timeseries, and compute:


*   number of confirmed COVID-19 cases across the globe
*   number of deaths due to COVID-19 (across the globe)
*   number of recovered patients across the globe



In [9]:
# YOUR CODE HERE
print(f"Number of confirmed cases: {int(confirmed.agg({'5/1/21':'sum'}).collect()[0][0])}")
print(f"Number of deaths: {int(deaths.agg({'5/1/21':'sum'}).collect()[0][0])}")
print(f"Number of recoverd patients: {int(recovered.agg({'5/1/21':'sum'}).collect()[0][0])}")

Number of confirmed cases: 152196159
Number of deaths: 3192930
Number of recoverd patients: 88919401


Consider the data points for March 1, 2020, and March 1, 2021, and filter out the geographical locations where less than 50 cases have been confirmed.
For the areas still taken into consideration after the filtering step, compute the ratio between number of deaths and number of confirmed cases.

In [39]:
# YOUR CODE HERE
deaths_select = deaths.select([col('Province/State').alias('state'), col('Country/Region').alias('country') ,(col('3/1/20') + col('3/1/21')).alias('num_dead')])

conf_select = confirmed.select([col('Province/State').alias('state'), col('Country/Region').alias('country') ,(col('3/1/20') + col('3/1/21')).alias('num_confirmed')])
conf_select = conf_select.filter(conf_select.num_confirmed >= 50)

joined_df = deaths_select.join(conf_select, ['state', 'country'], 'inner')
joined_df = joined_df.select(['state', 'country', (col('num_dead') / col('num_confirmed')).alias('dead_conf_ratio')])
joined_df.sort(joined_df.dead_conf_ratio.desc()).show(25)


+--------------------+--------------+--------------------+
|               state|       country|     dead_conf_ratio|
+--------------------+--------------+--------------------+
|            Tasmania|     Australia| 0.05555555555555555|
|               Hubei|         China| 0.05385093811547632|
|         Isle of Man|United Kingdom|   0.051440329218107|
|            Victoria|     Australia| 0.04002342834830144|
|         Nova Scotia|        Canada| 0.03958587088915956|
|              Quebec|        Canada|  0.0360633110690332|
|              Hainan|         China|0.032448377581120944|
|            Manitoba|        Canada| 0.02809305825547125|
|Australian Capita...|     Australia|0.025423728813559324|
|             Ontario|        Canada|0.022728530458249102|
|           Gibraltar|United Kingdom|0.021939136588818117|
|     Channel Islands|United Kingdom| 0.02129767211490837|
|       New Brunswick|        Canada|0.018867924528301886|
|           Hong Kong|         China|0.01817363922627080

Consider the data points for March 1, 2021, and May 1, 2021, in the timeseries, and filter out the geographical locations where less than 50 deaths have been confirmed (as of March 1, 2021).
For the areas still taken into consideration after the filtering step, compute the percent increase in cumulative deaths between the two dates.

In [45]:
# YOUR CODE HERE
dead_date_1 = deaths.select([col('Province/State').alias('state'), col('Country/Region').alias('country') ,col('3/1/20').alias('date_1_num_dead')])
dead_date_1 = dead_date_1.filter(dead_date_1.date_1_num_dead >= 50)
dead_date_2 = deaths.select([col('Province/State').alias('state'), col('Country/Region').alias('country') ,col('5/1/20').alias('date_2_num_dead')])
joined_df = dead_date_1.join(dead_date_2, ['state', 'country'])
joined_df = joined_df.select(['state', 'country', (col('date_2_num_dead') / col('date_1_num_dead')).alias('pct_increase')]).show()


+-----+-------+------------------+
|state|country|      pct_increase|
+-----+-------+------------------+
|Hubei|  China|1.6341905106845347|
+-----+-------+------------------+



Once you have working code for each cell above, **head over to Gradescope, read carefully the questions, and submit your solution for this Colab**!